In [209]:
#Libraries
import pandas as pd
import sqldf

### Data Structures
1. We are only concern about MTG Site Atribbutes
2. LMS Sites can be used too
3. No TOMS data

In [210]:
# Read and print data from all sites v1

all_sites = pd.read_excel('data/all_sites.xlsx')
all_sites_reduced_columns = [
    "Site ID",
    "LMS Site ID",
    "TOMS Site ID",
    "Velocity Site ID",
    "Site Name",
    "Site Status",
    "Site Type"
]
all_sites = all_sites[all_sites_reduced_columns]
all_sites = all_sites[(all_sites["Site Status"] == "ACTIVE")
                      & (all_sites["Site Type"] == "MINING")]
print(all_sites.shape)
display(all_sites.sample(5))


(220, 7)


,Site ID,LMS Site ID,TOMS Site ID,Velocity Site ID,Site Name,Site Status,Site Type
285,AUS053,AUS053,NaN,AU900,PERTH,ACTIVE,MINING
397,MEX041,MEX041,NaN,MX605,HERMOSILLO (SERVICE TCAR),ACTIVE,MINING
231,308,308,NaN,CA308,KEY LAKE,ACTIVE,MINING
320,CHI021,CHI021,NaN,CL308,EL ABRA,ACTIVE,MINING
452,CHI003,CHI003,CL304,CL304,SPENCE,ACTIVE,MINING


In [211]:
# Read and print data from mtg_site_attributes

mtg_site_attributes = pd.read_excel('data/mtg_site_attributes.xlsx')
mtg_site_attributes_reduced_columns = [
    "Code",
    "SiteID",
    "MineType",
    "SiteType",
    "Status",
    "Commodity",
    "Locale",
    "Organization",
    "Customer",
    "Name"
]
mtg_site_attributes = mtg_site_attributes[mtg_site_attributes_reduced_columns]
print(mtg_site_attributes.shape)
display(mtg_site_attributes.sample(5))

(1041, 10)


,Code,SiteID,MineType,SiteType,Status,Commodity,Locale,Organization,Customer,Name
435,SA1050,RSA002,Open Pit,Mine - Off Site Service (OTR),NaN,NaN,NaN,Farm Zonnebloem,Canyon Coal,Farm Zonnebloem
105,CA8361,836,Non Mining,Commercial - Off Site,Active,Steel,North America,Dofasco,ArcelorMittal Dofasco,Dofasco
728,UK200063,UK001,Non Mining,NaN,Active,NaN,Europe,Veolia UK,Veolia UK,Maresfield (Veolia)
91,CA3720,372,Open Pit,Mine - On Site Service (OTR),Active,Petroleum Products,North America,Basemine,Syncrude,Basemine
205,CA836635,836,Non Mining,Quarry - Off Site Service,Active,Aggregate,North America,Lafarge Canada,Lafarge Canada,Commissioner (Toronto)


In [212]:
lms_location_names = pd.read_excel('data/lms_location_names.xlsx')
print(lms_location_names.shape)
display(lms_location_names.sample(5))

(572, 2)


,LMS Site ID,Site Name
215,847,Kingston
528,MEX042,Planta de Renovado
556,887,Sudbury
346,990,Kal Tire Cirkular
123,130,Sechelt


### 1st Intermediate Table
The goal is to join all_sites_v1 with mtg_site_attributes with the key column: "Site ID" in case when more than 1 result for each site is shown. The better match will be returned

In [213]:
# New table only join (Query)
with open('querys/int_all_sites_attributes_match.sql', 'r') as fp:
    int_query = fp.read()

int_all_sites_attributes_match = sqldf.run(int_query)
print(int_all_sites_attributes_match.shape)
display(int_all_sites_attributes_match.sample(5))

(801, 3)


,Site ID,Site Name,MTG Name
587,UK001,ALFRETON UK,Whitemoss (Veolia)
111,COL002,CALENTURITAS,Calenturitas
65,AUS038,GLENCORE - GEORGE FISHER,Lady Loretta (Redpath)
491,UK001,ALFRETON UK,Leeds Accomodation Rd(Han
71,AUS049,KAL AUS PAYROLL,Head Office Payroll


In [214]:
# Now a matching score is computed by vectorizing each name
def match(ref_name: str, test_name: str) -> int:
    try:
        set_ref_name = set(ref_name.strip().lower().split())
        set_test_name = set(test_name.strip().lower().split())
        matchs = len(set_ref_name.intersection(set_test_name))
    except:
        return 0
    return matchs


int_all_sites_attributes_match["Score"] = int_all_sites_attributes_match.apply(
    lambda x: match(x["Site Name"], x["MTG Name"]), axis=1)
int_all_sites_attributes_match.sample(5)
int_all_sites_attributes_match = int_all_sites_attributes_match.loc[int_all_sites_attributes_match.groupby([
                                                                                                           "Site ID"])["Score"].idxmax()]
print(int_all_sites_attributes_match.shape)
display(int_all_sites_attributes_match.sample(5))

(162, 4)


,Site ID,Site Name,MTG Name,Score
299,RSA059,ALRODE BAKERS TRANSPORT,Bakers Transport,2
301,RSA069,BRITS OFFICE,Brits,1
37,386,386 KAL TIRE,Eagle Gold,0
20,333,KAL TIRE,Canadian Technical Suppor,0
277,RSA035,STEELPOORT,Steelpoort,1


### Final Modeled Table
In this table, only the raw tables (all_sites and lms_location_names) are joined with the previous intermediate one

In [215]:
with open('querys/modeled_all_sites.sql', 'r') as fp:
    final_query = fp.read()

modeled_all_sites = sqldf.run(final_query).set_index(['Site ID'])
print(modeled_all_sites.shape)
display(modeled_all_sites.sample(5))

modeled_all_sites.to_excel('modeled_all_sites_corrected_name.xlsx')

(220, 8)


,LMS Site ID,TOMS Site ID,Site Status,Site Type,all_sites_name,mtg_site_name,lms_site_name,corrected_site_name
Site ID,,,,,,,,
AUS055,AUS055,None,ACTIVE,MINING,KAL AUS TECHNICAL SERVICE,None,Kal Aus Technical Services,Kal Aus Technical Services
MEX029,MEX029,None,ACTIVE,MINING,NOCHE BUENA,Noche Buena,Noche Buena,Noche Buena
357,357,None,ACTIVE,MINING,357 MALARTIC,Canadian Malartic,Malartic,Canadian Malartic
AUS036,AUS036,None,ACTIVE,MINING,ERNEST HENRY MINING,Ernest Henry,Ernest Henry Mining,Ernest Henry
AUS058,AUS058,None,ACTIVE,MINING,MIDDLEMOUNT,Middlemount Coal,Middlemount,Middlemount Coal
